In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [69]:
!mkdir -p GuodongDrive
!google-drive-ocamlfuse GuodongDrive
import os
os.chdir("GuodongDrive")
os.chdir("ccf/carsale")
!ls

0825ccf_carsale.csv	   ccf_car_sales082601.csv  submit_example.csv
0825ccf_car_sales_lgb.csv  ccf_car_sales082602.csv  train_sales_data.csv
baseline082601.ipynb	   ccf_car_sales.csv	    train_search_data.csv
baseline1_copy.ipynb	   ccf_car_sales_lgb.csv    train_user_reply_data.csv
baseline1.ipynb		   evaluation_public.csv    Untitled0.ipynb
baseline2.ipynb		   GuodongDrive


In [0]:
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
import datetime
import time
import lightgbm as lgb
import xgboost as xgb

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [0]:
def mse(true_label,pre_label):
    squaredError = (true_label - pre_label)*(true_label - pre_label)
    return sum(squaredError) / len(squaredError)

In [0]:
train_sales  = pd.read_csv('train_sales_data.csv')
train_search = pd.read_csv('train_search_data.csv')
train_user   = pd.read_csv('train_user_reply_data.csv')

evaluation_public = pd.read_csv('evaluation_public.csv')
submit_example    = pd.read_csv('submit_example.csv')

In [0]:
m1_12    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12), 'salesVolume'].values
m1_11    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11), 'salesVolume'].values
m1_10    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10), 'salesVolume'].values
m1_09    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values
m1_08    = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==8) , 'salesVolume'].values

m1_12_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==12), 'salesVolume'].values * m1_12
m1_11_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==11), 'salesVolume'].values * m1_11
m1_10_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==10), 'salesVolume'].values * m1_10
m1_09_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==9) , 'salesVolume'].values * m1_09
m1_08_volum = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==8) , 'salesVolume'].values * m1_08

evaluation_public.loc[evaluation_public.regMonth==1, 'forecastVolum'] =  m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/16 + m1_08_volum/16

In [0]:
m16_1_2  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values
m16_1_3  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values
m16_1_4  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values
m16_1_5  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values

m16_2_3  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values
m16_2_4  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values
m16_2_5  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values
m16_2_6  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values

m16_3_4  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values
m16_3_5  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values
m16_3_6  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values
m16_3_7  = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==7) , 'salesVolume'].values

m17_1_2  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values
m17_1_3  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values
m17_1_4  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m17_1_5  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values


m17_2_3  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values
m17_2_4  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m17_2_5  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values
m17_2_6  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==6) , 'salesVolume'].values

m17_3_4  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m17_3_5  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values
m17_3_6  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==6) , 'salesVolume'].values
m17_3_7  = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==7) , 'salesVolume'].values

m16_1 = m16_1_2/2 + m16_1_3/4 + m16_1_4/8 + m16_1_5/8
m16_2 = m16_2_3/2 + m16_2_4/4 + m16_2_5/8 + m16_2_6/8
m16_3 = m16_3_4/2 + m16_3_5/4 + m16_3_6/8 + m16_3_7/8

m17_1 = m17_1_2/2 + m17_1_3/4 + m17_1_4/8 + m17_1_5/8
m17_2 = m17_2_3/2 + m17_2_4/4 + m17_2_5/8 + m17_2_6/8
m17_3 = m17_3_4/2 + m17_3_5/4 + m17_3_6/8 + m17_3_7/8

m1 = m16_1 * 0.4 +  m17_1 * 0.6
m2 = m16_2 * 0.4 +  m17_2 * 0.6
m3 = m16_3 * 0.4 +  m17_3 * 0.6

evaluation_public.loc[evaluation_public.regMonth==2, 'forecastVolum'] = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values / m1
evaluation_public.loc[evaluation_public.regMonth==3, 'forecastVolum'] = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values / m2
evaluation_public.loc[evaluation_public.regMonth==4, 'forecastVolum'] = train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values / m3

In [0]:
print(train_sales.loc[(train_sales.regMonth<=4) ,'salesVolume'].mean())
print(evaluation_public['forecastVolum'].mean())
evaluation_public[['id','forecastVolum']].round().astype(int).to_csv('ccf_car_sales.csv', index=False)

524.1121212121212
477.4628274411716


In [0]:
data = pd.concat([train_sales, evaluation_public], ignore_index=True)
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']

num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum']
cate_feat = ['bodyType', 'model', 'province']

for i in cate_feat:
    data[i] = data[i].astype('category')
features = num_feat + cate_feat

In [83]:
data['bodyType'] = pd.Categorical(data.bodyType).codes
data['province'] = pd.Categorical(data.province).codes
data['model'] = pd.Categorical(data.model).codes
data.head(3)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label
0,310000,2,0,13,0,1,2016,1479.0,11.0,106.0,292.0
1,530000,2,0,13,1,1,2016,1594.0,11.0,106.0,466.0
2,150000,2,0,13,2,1,2016,1479.0,11.0,106.0,257.0


不同月份 的省份所卖的总车辆、在这一年中占的比重

不同月份，的车型所卖的总车辆，在这一年中占的比重

不同月份，的车类型所卖的总车辆，在这一年中占的比重


不同的季度，同上

省 车型 车类型 时间

 省 车类型 时间
 
 省 车型 时间
 
 全国 车型 时间
 
全国 车类型 时间

↑表示group by的参数。

是否有月份异常需要单独拿出来训练

当月是哪个季节？？【还未加入】

当月是上半年还是下半年？？【还未加入】

相邻年，一阶差分，一阶比值

相邻月份，一阶差分，一阶比值

历史月份销量比例，考虑时间衰减，月份越近占比越高*销量

## 时间处理

In [0]:
data['date'] = (np.array(data.regYear.astype(str),dtype='datetime64[Y]') 
 + np.array(data.regMonth-1,dtype='timedelta64[M]'))
data['date2'] = pd.to_datetime(data['date'])
data['date2']

In [0]:
data['date2'].head(5)

0       2016-01-01
1       2016-01-01
2       2016-01-01
3       2016-01-01
4       2016-01-01
5       2016-01-01
6       2016-01-01
7       2016-01-01
8       2016-01-01
9       2016-01-01
10      2016-01-01
11      2016-01-01
12      2016-01-01
13      2016-01-01
14      2016-01-01
15      2016-01-01
16      2016-01-01
17      2016-01-01
18      2016-01-01
19      2016-01-01
20      2016-01-01
21      2016-01-01
22      2016-01-01
23      2016-01-01
24      2016-01-01
25      2016-01-01
26      2016-01-01
27      2016-01-01
28      2016-01-01
29      2016-01-01
           ...    
36930   2018-04-01
36931   2018-04-01
36932   2018-04-01
36933   2018-04-01
36934   2018-04-01
36935   2018-04-01
36936   2018-04-01
36937   2018-04-01
36938   2018-04-01
36939   2018-04-01
36940   2018-04-01
36941   2018-04-01
36942   2018-04-01
36943   2018-04-01
36944   2018-04-01
36945   2018-04-01
36946   2018-04-01
36947   2018-04-01
36948   2018-04-01
36949   2018-04-01
36950   2018-04-01
36951   2018

In [0]:


data['regMonth'] = data['regMonth'].apply(lambda x: int(x) )
data['regYear'] = data['regYear'].apply(lambda x: int(x) )
data['date'] = datetime.date.__new__(data['regYear'],data['regMonth'])
# data['regYear'].apply(lambda x: str(x) ) +data['regMonth'].apply(lambda x: str(x) )

# data['date'] =pd.to_datetime(data['date'])
data['date']

TypeError: ignored

In [0]:
test_index = (data['label'].isnull()) | (data['label'] == -1)
train_data = data[~test_index].reset_index(drop=True)

train_x = train_data[features]
train_y = train_data['label']


In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
parameters = {
	'num_leaves':[64], 
	'reg_alpha':[0.], 
 	'reg_lambda':[ 2],
   'max_depth':[-1],
   'min_child_samples':[50],
    'subsample':[0.7],
     'n_estimators' : [1500],
    'subsample_freq': [1]
 
}
lgb_model = lgb.LGBMRegressor(
    num_leaves=32, reg_alpha=0., reg_lambda=0.01, objective='mse', metric='mae',
    max_depth=-1, learning_rate=0.05, min_child_samples=20,
    n_estimators=1000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,n_jobs=-1
)

clf = GridSearchCV( lgb_model, param_grid= parameters, scoring="neg_mean_squared_error",cv=5, verbose=10 )
clf.fit(train_x, train_y) #你和模型
clf.best_params_


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=1, score=-83915.506, total=   6.2s
[CV] max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s


[CV]  max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=1, score=-36829.321, total=   6.4s
[CV] max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=1 


...

[CV]  max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=2, score=-30493.695, total=   8.0s
[CV] max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=2 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.1min remaining:    0.0s


[CV]  max_depth=-1, min_child_samples=50, n_estimators=1500, num_leaves=64, reg_alpha=0.0, reg_lambda=2, subsample=0.7, subsample_freq=2, score=-85413.360, total=   8.1s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.2min finished


{'max_depth': -1,
 'min_child_samples': 50,
 'n_estimators': 1500,
 'num_leaves': 64,
 'reg_alpha': 0.0,
 'reg_lambda': 2,
 'subsample': 0.7,
 'subsample_freq': 1}

In [0]:
# def get_predict_w(model, data, label='label', feature=[], cate_feature=[], random_state=2018, n_splits=5,
#                   model_type='lgb'):
def get_predict_w(model, data, label='label', feature=[], random_state=2018, n_splits=5,
                  model_type='lgb'):
    if 'sample_weight' not in data.keys():
        data['sample_weight'] = 1
    model.random_state = random_state
    predict_label = 'predict_' + label
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    data[predict_label] = 0
    test_index = (data[label].isnull()) | (data[label] == -1)
    train_data = data[~test_index].reset_index(drop=True)
    test_data = data[test_index]

    for train_idx, val_idx in kfold.split(train_data):
        model.random_state = model.random_state + 1

        train_x = train_data.loc[train_idx][feature]
        train_y = train_data.loc[train_idx][label]

        test_x = train_data.loc[val_idx][feature]
        test_y = train_data.loc[val_idx][label]
        if model_type == 'lgb':
            try:
                model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
                          eval_metric='mae',
                          # callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                          categorical_feature=cate_feature,
                          sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=100)
            except:
                model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
                          eval_metric='mae',
                          # callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                          # categorical_feature=cate_feature,
                          sample_weight=train_data.loc[train_idx]['sample_weight'],
                          verbose=100)
        elif model_type == 'ctb':
            model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
                      # eval_metric='mae',
                      # callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                      cat_features=cate_feature,
                      sample_weight=train_data.loc[train_idx]['sample_weight'],
                      verbose=100)
        train_data.loc[val_idx, predict_label] = model.predict(test_x)
        if len(test_data) != 0:
            test_data[predict_label] = test_data[predict_label] + model.predict(test_data[feature])
    test_data[predict_label] = test_data[predict_label] / n_splits
    #print(mse(train_data[label], train_data[predict_label]))
    print(mse(train_data[label], train_data[predict_label]) * 5, train_data[predict_label].mean(),
          test_data[predict_label].mean())

    return pd.concat([train_data, test_data], sort=True, ignore_index=True), predict_label

In [0]:
def get_predict_w2(model, data, label='label', feature=[], cate_feature=[], random_state=2018, n_splits=5,
                  model_type='xgb'):
    if 'sample_weight' not in data.keys():
        data['sample_weight'] = 1
    model.random_state = random_state
    predict_label = 'predict_' + label
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    data[predict_label] = 0
    test_index = (data[label].isnull()) | (data[label] == -1)
    train_data = data[~test_index].reset_index(drop=True)
    test_data = data[test_index]

    for train_idx, val_idx in kfold.split(train_data):
        model.random_state = model.random_state + 1

        train_x = train_data.loc[train_idx][feature]
        train_y = train_data.loc[train_idx][label]

        test_x = train_data.loc[val_idx][feature]
        test_y = train_data.loc[val_idx][label]
        if model_type == 'lgb':
#             try:
          model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=200,
                    eval_metric='mae',
                    # callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                    categorical_feature=cate_feature,
                    sample_weight=train_data.loc[train_idx]['sample_weight'],
                    verbose=100)
#             except:
#                 model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                           eval_metric='mae',
#                           # callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
#                           # categorical_feature=cate_feature,
#                           sample_weight=train_data.loc[train_idx]['sample_weight'],
#                           verbose=100)
        elif model_type == 'xgb':
            model.fit(train_x, train_y, eval_set=[(test_x, test_y)], 
                      #early_stopping_rounds=200,
                       eval_metric='mae',
                      # callbacks=[lgb.reset_parameter(learning_rate=lambda iter: max(0.005, 0.5 * (0.99 ** iter)))],
                      #cat_features=cate_feature,
                      sample_weight=train_data.loc[train_idx]['sample_weight'],
                      verbose=200)
        train_data.loc[val_idx, predict_label] = model.predict(test_x)
        if len(test_data) != 0:
            test_data[predict_label] = test_data[predict_label] + model.predict(test_data[feature])
    test_data[predict_label] = test_data[predict_label] / n_splits
    #print(mse(train_data[label], train_data[predict_label]))
    print(mse(train_data[label], train_data[predict_label]) * 5, train_data[predict_label].mean(),
          test_data[predict_label].mean())

    return pd.concat([train_data, test_data], sort=True, ignore_index=True), predict_label

In [86]:
xgb_model = xgb.XGBRegressor(max_deepth =4,num_boost_round=1000)
data, predict_label = get_predict_w2(xgb_model, data, label='label',
                                    feature=features, cate_feature=cate_feat,
                                    random_state=2019, n_splits=5)
data['xgb'] = data[predict_label]

data['forecastVolum'] = data['xgb'].apply(lambda x: 0 if x < 0 else x)

[08:41:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:534.119
[99]	validation_0-mae:260.379
[08:41:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:548.882
[99]	validation_0-mae:268.88
[08:41:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:561.572
[99]	validation_0-mae:275.77
[08:41:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:544.121
[99]	validation_0-mae:261.847
[08:41:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:552.33
[99]	validation_0-mae:270.825
1077519.25908456 608.5443903135691 219.49841039964318


In [78]:
lgb_model = lgb.LGBMRegressor(
    num_leaves=32, reg_alpha=0., reg_lambda=0.01, objective='mse', metric='mae',
    max_depth=-1, learning_rate=0.05, min_child_samples=20,
    n_estimators=50000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
)


data, predict_label = get_predict_w(lgb_model, data, label='label',
                                    feature=features, 
                                    random_state=2019, n_splits=5)

data['lgb'] = data[predict_label]

data['forecastVolum'] = data['lgb'].apply(lambda x: 0 if x < 0 else x)


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 221.065
[200]	valid_0's l1: 184.071
[300]	valid_0's l1: 167.892
[400]	valid_0's l1: 157.69
[500]	valid_0's l1: 150.949
[600]	valid_0's l1: 145.815
[700]	valid_0's l1: 141.164
[800]	valid_0's l1: 137.39
[900]	valid_0's l1: 133.801
[1000]	valid_0's l1: 131.101
[1100]	valid_0's l1: 128.944
[1200]	valid_0's l1: 126.845
[1300]	valid_0's l1: 124.857
[1400]	valid_0's l1: 123.497
[1500]	valid_0's l1: 121.892
[1600]	valid_0's l1: 120.371
[1700]	valid_0's l1: 119.234
[1800]	valid_0's l1: 118.036
[1900]	valid_0's l1: 117.203
[2000]	valid_0's l1: 116.285
[2100]	valid_0's l1: 115.652
[2200]	valid_0's l1: 115.019
[2300]	valid_0's l1: 113.983
[2400]	valid_0's l1: 113.232
[2500]	valid_0's l1: 112.48
[2600]	valid_0's l1: 111.937
[2700]	valid_0's l1: 111.54
[2800]	valid_0's l1: 110.928
[2900]	valid_0's l1: 110.412
[3000]	valid_0's l1: 109.988
[3100]	valid_0's l1: 109.699
[3200]	valid_0's l1: 109.239
[3300]	valid_0's l1: 1

KeyboardInterrupt: ignored

In [0]:
data[data.label.isnull()][['id', 'forecastVolum']].round().astype(int).to_csv('ccf_car_sales082602.csv', index=False)

In [0]:
train_x,train_y,test_x,test_y  = train_test_split( X, y, test_size=0.33, random_state=42)